# Projeto - Análise do preço das casas de São Paulo

### Autor: Guilherme Iram
### Fonte dos dados: https://www.kaggle.com/datasets/kaggleshashankk/house-price-data-of-sao-paulo



# Informações sobre o dataset
- Street (Rua) - Localização da rua a qual a casa pertence
- City (Cidade) - Cidade que a casa está localizada
- Propertycard__detailvalue (Área) - Área em metros quadrados da casa
- Quartos (Quartos) - Número de quartos da casa
- Banheiros (Banheiros) - Número de banheiros da casa
- Vagas (Vagas) - Vagas de estacionamento por veículo
- Price (Custo) - Preço da casa em BRL (real brasileiro)

## Objetivos da análises:
- Baseado nas características quantitativas do dataset, fazer uma análise visual desses atributos e propor um modelo de regressão.
- Sugerir uma previsão do preço da casa baseado nas características desejadas pelo cliente.

## 1 - Carregando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import statistics as st

In [2]:
import warnings
warnings.filterwarnings('ignore')

## 2 - Carregando o dataset, tratamento & análises prévias

In [3]:
df_sp = pd.read_excel("./dados/SaoPaulo.xlsx")

In [4]:
df_sp.head()

,Street,City,propertycard__detailvalue,quartos,banheiros,vagas,price
0,"Avenida Itacira, 255","Planalto Paulista, São Paulo",1000,4,8,6,R$ 7.000.000
1,"Rua Aurelia Perez Alvarez, 42","Jardim dos Estados, São Paulo",469-524,3-6,3-6,4,R$ 3.700.000
2,Rua Alba Valdez,"Jardim Reimberg, São Paulo",125,4,3,2,R$ 380.000
3,"Jardim Morumbi, São Paulo",SP,310,3,2,4,R$ 685.000
4,"Rua Tobias Barreto, 195","Mooca, São Paulo",100,3,2,2,R$ 540.000


In [5]:
dict_rename = {'Street': 'rua',
              'City': 'cidade',
              'propertycard__detailvalue': 'area_metro_quadrado',
              'quartos': 'quartos',
              'banheiros': 'banheiros',
              'vagas': 'vagas',
              'price': 'preco'}
df_sp.rename(columns=dict_rename, inplace=True)

In [6]:
df_sp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10033 entries, 0 to 10032
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   rua                  10033 non-null  object
 1   cidade               10033 non-null  object
 2   area_metro_quadrado  10033 non-null  object
 3   quartos              10033 non-null  object
 4   banheiros            10033 non-null  object
 5   vagas                10033 non-null  object
 6   preco                10033 non-null  object
dtypes: object(7)
memory usage: 548.8+ KB


 - Necessidade de efetuar limpesa e tratamento de dados 
 - Limpar valores quantitativos, como os metros quadrados, número de quartos, banheiros e remover o cifrão do preço
 - Sem dados faltantes

## 3 - Limpeza dos dados

### 3.1 - Tratando a coluna 'preco'

In [7]:
df_sp["preco"].unique()

array(['R$ 7.000.000', 'R$ 3.700.000', 'R$ 380.000', ..., 'R$ 553.000',
       'R$ 951.000', 'R$ 2.200\n                      /Mês'], dtype=object)

In [8]:
df_sp["preco"].str.split(expand=True)

,0,1,2
0,R$,7.000.000,None
1,R$,3.700.000,None
2,R$,380.000,None
3,R$,685.000,None
4,R$,540.000,None
...,...,...,...
10028,R$,665.000,None
10029,R$,2.300.000,None
10030,R$,1.050.000,None
10031,R$,2.200,/Mês


In [9]:
df_sp["preco"].str.split(expand = True)[2].unique()

array([None, '/Mês', '/Ano', '/Dia'], dtype=object)

In [10]:
df_sp[["moeda", "preco_anuncio", "tipo_anuncio"]] = df_sp["preco"].str.split(expand=True)

In [11]:
df_sp["moeda"].unique()

array(['R$'], dtype=object)

In [12]:
mask_none = df_sp["tipo_anuncio"].isnull()
df_sp_filter = df_sp[mask_none]

In [13]:
df_sp_filter["preco_anuncio_float"] = df_sp_filter["preco_anuncio"].str.replace(".","").astype(float)
df_sp_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   rua                  9181 non-null   object 
 1   cidade               9181 non-null   object 
 2   area_metro_quadrado  9181 non-null   object 
 3   quartos              9181 non-null   object 
 4   banheiros            9181 non-null   object 
 5   vagas                9181 non-null   object 
 6   preco                9181 non-null   object 
 7   moeda                9181 non-null   object 
 8   preco_anuncio        9181 non-null   object 
 9   tipo_anuncio         0 non-null      object 
 10  preco_anuncio_float  9181 non-null   float64
dtypes: float64(1), object(10)
memory usage: 860.7+ KB


### 3.2 - Tratando a coluna 'area_metro_quadrado'

- Aqui será feito uma convenção, para todos os valores no formato 'x-y', será usado o x para substituir a expressão.
- Isso vale pras demais colunas.

In [14]:
df_sp_filter["area_metro_quadrado"].unique()[:10]

array(['1000', '469-524', '125', '310', '100', '440', '145', '150',
       '46-50', '114'], dtype=object)

In [15]:
df_sp_filter["area_metro_quadrado"].str.split("-" ,expand=True)

,0,1
0,1000,None
1,469,524
2,125,None
3,310,None
4,100,None
...,...,...
10027,262,None
10028,90,None
10029,600,None
10030,200,None


In [16]:
df_sp_filter["area_metro_quadrado_int"] = df_sp_filter["area_metro_quadrado"].str.split("-" ,expand=True)[0].astype(int)

### 3.3 - Tratando a coluna 'quartos'

In [17]:
df_sp_filter["quartos"].unique()

array(['4', '3-6', '3', '2', '7', '2-3', '5', '6', '1', '8', '11', '10',
       '9', '13', '12', '16'], dtype=object)

In [18]:
df_sp_filter["quartos"].str.split("-", expand=True)

,0,1
0,4,None
1,3,6
2,4,None
3,3,None
4,3,None
...,...,...
10027,3,None
10028,2,None
10029,8,None
10030,3,None


In [19]:
df_sp_filter["quartos"].str.split("-", expand=True)[0]

0        4
1        3
2        4
3        3
4        3
        ..
10027    3
10028    2
10029    8
10030    3
10032    3
Name: 0, Length: 9181, dtype: object

In [20]:
df_sp_filter["quartos_int"] = df_sp_filter["quartos"].str.split("-", expand=True)[0].astype(int)

### 3.4 - Tratando a coluna 'banheiros'

In [21]:
df_sp_filter["banheiros"].unique()

array(['8', '3-6', '3', '2', '4', '1', '5', '7', '6', '9', '10', '11',
       '12', '40', '14'], dtype=object)

In [22]:
df_sp_filter["banheiros"].str.split('-', expand=True)

,0,1
0,8,None
1,3,6
2,3,None
3,2,None
4,2,None
...,...,...
10027,6,None
10028,2,None
10029,7,None
10030,3,None


In [23]:
df_sp_filter["banheiros"].str.split('-', expand=True)[0]

0        8
1        3
2        3
3        2
4        2
        ..
10027    6
10028    2
10029    7
10030    3
10032    2
Name: 0, Length: 9181, dtype: object

In [24]:
df_sp_filter["banheiros_int"] = df_sp_filter["banheiros"].str.split('-', expand=True)[0].astype(int) 

### 3.5 - Tratando a coluna 'vagas'

In [25]:
df_sp_filter["vagas"].unique()

array(['6', '4', '2', '1', '3', '5', '1-2', '8', '7', '10', '9', '20',
       '13', '25', '14', '12', '11', '38', '30', '26', '15', '24', '16',
       '18', '50'], dtype=object)

In [26]:
df_sp_filter["vagas"].str.split("-", expand=True)

,0,1
0,6,None
1,4,None
2,2,None
3,4,None
4,2,None
...,...,...
10027,4,None
10028,2,None
10029,5,None
10030,6,None


In [27]:
df_sp_filter["vagas"].str.split("-", expand=True)[0]

0        6
1        4
2        2
3        4
4        2
        ..
10027    4
10028    2
10029    5
10030    6
10032    2
Name: 0, Length: 9181, dtype: object

In [28]:
df_sp_filter["vagas_int"] = df_sp_filter["vagas"].str.split("-", expand=True)[0].astype(int)

### 3.6 - Cotemplando as mudanças & salvando novo xlsx

In [29]:
df_sp_filter.head()

,rua,cidade,area_metro_quadrado,quartos,banheiros,vagas,preco,moeda,preco_anuncio,tipo_anuncio,preco_anuncio_float,area_metro_quadrado_int,quartos_int,banheiros_int,vagas_int
0,"Avenida Itacira, 255","Planalto Paulista, São Paulo",1000,4,8,6,R$ 7.000.000,R$,7.000.000,None,7000000.0,1000,4,8,6
1,"Rua Aurelia Perez Alvarez, 42","Jardim dos Estados, São Paulo",469-524,3-6,3-6,4,R$ 3.700.000,R$,3.700.000,None,3700000.0,469,3,3,4
2,Rua Alba Valdez,"Jardim Reimberg, São Paulo",125,4,3,2,R$ 380.000,R$,380.000,None,380000.0,125,4,3,2
3,"Jardim Morumbi, São Paulo",SP,310,3,2,4,R$ 685.000,R$,685.000,None,685000.0,310,3,2,4
4,"Rua Tobias Barreto, 195","Mooca, São Paulo",100,3,2,2,R$ 540.000,R$,540.000,None,540000.0,100,3,2,2


In [30]:
df_to_save = df_sp_filter[["area_metro_quadrado_int", "quartos_int", "banheiros_int", "vagas_int", "preco_anuncio_float"]]
df_to_save.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9181 entries, 0 to 10032
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   area_metro_quadrado_int  9181 non-null   int32  
 1   quartos_int              9181 non-null   int32  
 2   banheiros_int            9181 non-null   int32  
 3   vagas_int                9181 non-null   int32  
 4   preco_anuncio_float      9181 non-null   float64
dtypes: float64(1), int32(4)
memory usage: 286.9 KB


In [31]:
df_to_save.to_csv("./dados/SaoPauloClean.csv")